In [1]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from collections import deque
import matplotlib.pyplot as plt

In [2]:
class A2C(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(A2C, self).__init__()
        self.common = Dense(64, activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        self.actor  = Dense(action_size, activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        self.critic = Dense(1,kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        
    def call(self, x):
        x      = self.common(x)
        policy = self.actor(x)
        value  = self.critic(x)
        return policy, value

In [3]:
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        
        self.model = A2C(self.state_size,self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.model.build(input_shape=(None, self.state_size))
        self.model.summary()
        
        
    def get_action(self, state):
        state = np.reshape(state, [1, self.state_size])
        policy, _ = self.model(state)
        policy = np.array(policy[0])
        return np.random.choice(self.action_size, 1, p=policy)[0]
        
    def train_model(self, state, action, reward, next_state, done):
        state = np.reshape(state, [1, self.state_size])
        next_state = np.reshape(next_state, [1, self.state_size])
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            policy, value      = self.model(state)
            _,      next_value = self.model(next_state)
            target = reward + (1 - done) * self.discount_factor * next_value[0]
            
            # For policy network
            one_hot_action = tf.one_hot([action], self.action_size)
            action_prob = tf.reduce_sum(one_hot_action * policy, axis=1)
            cross_entropy = - tf.math.log(action_prob + 1e-5)
            advantage = tf.stop_gradient(target - value[0])
            actor_loss = tf.reduce_mean(cross_entropy * advantage)
            
            # For value network
            critic_loss = 0.5 * tf.square(tf.stop_gradient(target) - value[0])
            critic_loss = tf.reduce_mean(critic_loss)
            
            # integrate losses
            loss = 0.2 * actor_loss + critic_loss
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return np.array(loss)

    def save_model(self):
        self.model.save_weights('./save_model/cartpole_a2c_TF', save_format='tf')

In [4]:
%matplotlib tk

ENV_NAME = 'CartPole-v1'
EPISODES = 3000
END_SCORE = 300

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = A2CAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        
        while not done:
            # env.render()

            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            loss = agent.train_model(state, action, reward, next_state, done)
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | loss: {:.4f}'.format(e, score_avg, np.mean(loss_list)))

                # Save data for plot
                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))
                
                # View data
                plt.subplot(211)
                plt.plot(episodes, scores_avg, 'b')
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title('cartpole A2C TF')
                plt.subplot(212)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode'); plt.ylabel('loss'); plt.grid()
                plt.savefig('./save_model/cartpole_a2c_TF.png')

                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if end == True:
            np.save('./save_model/data/cartpole_a2c_TF_epi',episodes)
            np.save('./save_model/data/cartpole_a2c_TF_score',scores)
            np.save('./save_model/data/cartpole_a2c_TF_loss',losses)
            env.close()
            print("End")
            break

i: 2058 | score avg 41.94 | loss: 0.8425
epi: 2059 | score avg 42.15 | loss: 1.5351
epi: 2060 | score avg 41.83 | loss: 1.2945
epi: 2061 | score avg 40.65 | loss: 3.3010
epi: 2062 | score avg 39.88 | loss: 3.6161
epi: 2063 | score avg 40.00 | loss: 2.0645
epi: 2064 | score avg 38.30 | loss: 5.4305
epi: 2065 | score avg 38.17 | loss: 1.4735
epi: 2066 | score avg 43.95 | loss: 0.2623
epi: 2067 | score avg 43.76 | loss: 2.0440
epi: 2068 | score avg 42.08 | loss: 3.7134
epi: 2069 | score avg 48.87 | loss: 0.5680
epi: 2070 | score avg 48.48 | loss: 1.1247
epi: 2071 | score avg 48.04 | loss: 0.9885
epi: 2072 | score avg 49.13 | loss: 1.1332
epi: 2073 | score avg 50.32 | loss: 0.9148
epi: 2074 | score avg 50.19 | loss: 1.4599
epi: 2075 | score avg 50.77 | loss: 0.6384
epi: 2076 | score avg 53.09 | loss: 1.5388
epi: 2077 | score avg 50.68 | loss: 4.1166
epi: 2078 | score avg 49.41 | loss: 3.3750
epi: 2079 | score avg 51.27 | loss: 0.2794
epi: 2080 | score avg 50.25 | loss: 3.9757
epi: 2081 | s

KeyboardInterrupt: 